In [1]:
import geopandas as gpd
import pandas as pd

## Open State data

In [2]:
file_path = "data/cb_2024_us_county_500k/cb_2024_us_county_500k.shp"
counties_gdf = gpd.read_file(file_path)

## Get Ethnicity amounts

In [3]:
acs_ethnicity = pd.read_csv(
    "data/ACSDT5Y2023.B04006_2025-05-02T185520/ACSDT5Y2023.B04006-Data.csv",
    low_memory=False,
)

#### Get Column names

In [4]:
acs_column_names = pd.read_csv(
    "data/ACSDT5Y2023.B04006_2025-05-02T185520/ACSDT5Y2023.B04006-Column-Metadata.csv"
)

In [5]:
ethnicity_cols = list(
    acs_column_names[
        (acs_column_names["Column Name"] == "B04006_033E")
        | (acs_column_names["Column Name"] == "B04006_001E")
        | (acs_column_names["Column Name"] == "B04006_039E")
        | (acs_column_names["Column Name"] == "B04006_047E")
        | (acs_column_names["Column Name"] == "B04006_059E")
        | (acs_column_names["Column Name"] == "B04006_065E")
        | (acs_column_names["Column Name"] == "B04006_089E")
    ]["Column Name"]
)

#### Back to Data

In [6]:
rename_cols = {
    "Estimate!!Total:!!Danish": "DANISH",
    "Estimate!!Total:!!Finnish": "FINNISH",
    "Estimate!!Total:!!Icelander": "ICELANDER",
    "Estimate!!Total:!!Norwegian": "NORWEGIAN",
    "Estimate!!Total:!!Swedish": "SWEDISH",
    "Estimate!!Total:!!Scandinavian": "SCANDINAVIAN",
    "Geography": "GEOIDFQ",
    "Estimate!!Total:": "TOTAL",
}

In [7]:
ethnicity_per_county_df = acs_ethnicity[["GEO_ID", *ethnicity_cols]]
ethnicity_per_county_df.columns = ethnicity_per_county_df.iloc[0]
ethnicity_per_county_df = ethnicity_per_county_df[1:]
ethnicity_per_county_df = ethnicity_per_county_df.rename(columns=rename_cols)

In [8]:
ancestry_cols = list(rename_cols.values())[:-2]

In [9]:
ethnicity_per_county_df[ancestry_cols] = ethnicity_per_county_df[ancestry_cols].astype(
    int
)

In [10]:
ethnicity_per_county_df["ancestry_total"] = ethnicity_per_county_df[ancestry_cols].sum(
    axis=1
)
ethnicity_per_county_df["ancestry_total_percent"] = ethnicity_per_county_df[
    "ancestry_total"
] / ethnicity_per_county_df["TOTAL"].astype(int)

## Merge Data

In [11]:
ethnicity_per_county_df["ancestry_countries_1_percent"] = ethnicity_per_county_df[
    ethnicity_per_county_df["ancestry_total_percent"] >= 0.01
][ancestry_cols].idxmax(axis=1)
ethnicity_per_county_df["ancestry_countries_2_percent"] = ethnicity_per_county_df[
    ethnicity_per_county_df["ancestry_total_percent"] >= 0.02
][ancestry_cols].idxmax(axis=1)
ethnicity_per_county_df["ancestry_countries_5_percent"] = ethnicity_per_county_df[
    ethnicity_per_county_df["ancestry_total_percent"] >= 0.05
][ancestry_cols].idxmax(axis=1)

In [12]:
ethnicity_per_county_gdf = counties_gdf.merge(
    ethnicity_per_county_df, on="GEOIDFQ", how="left"
)

In [13]:
ethnicity_per_county_gdf["county_id"] = ethnicity_per_county_gdf["GEOIDFQ"].str[9:]
ethnicity_per_county_gdf["state_id"] = ethnicity_per_county_gdf["GEOIDFQ"].str[9:11]

In [14]:
ethnicity_per_county_gdf = ethnicity_per_county_gdf.to_crs(9311)
ethnicity_per_county_gdf.to_file("data/nordic_ancestry_per_county.gpkg")